In [410]:
import random
import imageio
import numpy as np
from PIL import Image
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential, Model

In [411]:
def make_trainable(net, val):
    net.model.trainable = val
    for l in net.model.layers:
        l.trainable = val
    net.cmpile()

In [412]:
class Dataset():
    x = None
    y = None
    W = None
    
    def __init__(self):
        return
    
    def load_data(self, data_range=10000):
        dots_x = []
        for i in range(data_range):
            x = random.gauss(0, data_range)
            dots_x.append((x, x**2)) # square
            dots_x.append((x, random.gauss(x**2, data_range**2.1))) # less than square
            #dots_x.append((x**r, x**(2*r*(1+random.gauss(0.5, 0.25)/20)))) # more than square
        dots_x = np.array(dots_x)
        dots_y = np.array([1 if x[0]**2 == x[1] else 0 for x in dots_x])
        self.x, self.y = dots_x, dots_y
        return dots_x, dots_y
    
    def load_weights(self, default_weight=0.12):
        W = self.y.copy().astype(float)
        W[W == 0] = 0.12
        self.W = W
        return W
        
    @staticmethod
    def load_random(n_dim=5, n_samples=16):
        V = np.random.normal(size=(n_samples, n_dim))
        return V
    

In [413]:
class Plot:
    name = ""
    images = []
    threshold = 0.0
    
    def __init__(self, name, threshold=0.6):
        self.name = name
        self.threshold = threshold
        self.images = []
    
    @staticmethod
    def parabola_plot(ax, xrange):
        x = np.linspace(xrange, 1)
        y = x*x
        plt.plot(x, y)
    
    @staticmethod
    def dots_plot(ax, dots_x, dots_y, color):
        ax.scatter(dots_x, dots_y, color=color, alpha=0.15)
        plt.plot()
    
    def picture(self, dots, predictions, title=''):
        predictions = predictions.reshape(predictions.shape[0])
        dots_x = dots.T[0]
        fig = plt.figure()
        ax1 = fig.add_subplot(111)
        ax1.set(title=title)
        #ax2 = ax1.twinx()
        plt.grid(axis='both')
        xrange = (dots_x.min()*1.1, dots_x.max()*1.1)
        self.parabola_plot(ax1, xrange)
        right_dots = dots[np.where(predictions >= self.threshold)]
        wrong_dots = dots[np.where(predictions < self.threshold)]
        #ax1.set_ylim([0, 2e+8])
        self.dots_plot(ax1, wrong_dots.T[0], wrong_dots.T[1], color='r')
        self.dots_plot(ax1, right_dots.T[0], right_dots.T[1], color='g')
        
    def add_to_gif(self, dots_x, predictions, title=''):
        self.picture(dots_x, predictions, title=title)
        plt.savefig(self.name+'.png')
        plt.close()
        image = Image.open(self.name+'.png')
        ar = np.asarray(image)
        self.images.append(ar)
        
    def save_gif(self):
        kargs = { 'duration': 0.1 }
        imageio.mimsave(self.name+'.gif', self.images, None, **kargs)
        

In [414]:
class Gen:
    model = None
    
    def __init__(self):
        model = Sequential([Dense(15, activation='relu', kernel_initializer='he_uniform', input_dim=5),
                            Dense(2, activation='linear')
        ])
        self.model = model
        
    def predict(self, dots_x):
        return self.model.predict(dots_x)
    
    def cmpile(self):
        return
    
    
class Dsc:
    model = None
    
    def __init__(self):
        model = Sequential([Dense(25, activation='relu', kernel_initializer='he_uniform', input_dim=2),
                            Dense(1, activation='sigmoid')
        ])
        self.model = model
    
    def cmpile(self, lr=0.0001):
        self.model.compile(loss='binary_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(lr),
                           metrics=['accuracy'])
        
    def fit(self, dots_x, dots_y, weights=None, epochs=1, validation_split=0.15, plot=False):
        if plot:
            img = Plot('discriminator_fit')
            for i in range(epochs):
                self.model.fit(dots_x, 
                               dots_y, 
                               epochs=1, 
                               sample_weight=weights)
                img.add_to_gif(dots_x, self.model.predict(dots_x), title='Epoch %d' % i)
                #print(dots_x[15], self.model.predict(dots_x)[15])
                img.save_gif()                        
        else:
            self.model.fit(dots_x, 
                           dots_y, 
                           epochs=epochs, 
                           sample_weight=weights, 
                           validation_split=validation_split)
        
# Raw class, doesn't work   
class Gan:
    gen = None
    dsc = None
    model = None
    
    def __init__(self, gen, dsc, n_dim=5):
        make_trainable(dsc, False)
        self.gen = gen
        self.dsc = dsc
        # connect them
        model = Sequential()
        # add generator
        model.add(gen.model)
        # add the discriminator
        model.add(dsc.model)
        self.model = model
    
    def cmpile(self, lr=0.001):
        self.model.compile(loss='binary_crossentropy',
                           optimizer=tf.keras.optimizers.Adam(lr),
                           metrics = ['accuracy'])
        
    def fit(self, dots_x, dots_y, epochs=1):
        self.model.fit(dots_x, dots_y, epochs=epochs)
        

In [415]:
def define_gan(generator, discriminator):
    # make weights in the discriminator not trainable
    discriminator.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(generator)
    # add the discriminator
    model.add(discriminator)
    # при замене оптимизатора всё слетает ???
    model.compile(loss='binary_crossentropy', optimizer='adam')
    return model

In [416]:
generator = Gen()
discriminator = Dsc()
gan = define_gan(generator.model, discriminator.model)

In [417]:
Data = Dataset()
Data.load_data()
weights = Data.load_weights()

In [418]:
discriminator.cmpile()
make_trainable(discriminator, True)
print(discriminator.model.trainable)
discriminator.fit(Data.x, Data.y, epochs=2, plot=True)

True
20000/20000 [==============================] - 4s 208us/sample - loss: 35803.9394 - acc: 0.5671


In [420]:
n_epochs = 3
#gan.compile()
discriminator.cmpile()

for i in range(n_epochs):
    V = Data.load_random(n_samples=1024)
    generated = generator.predict(V)
    make_trainable(discriminator, False)
    gan.fit(V, np.ones(V.shape[0]))
    make_trainable(discriminator, True)
    discriminator.fit(generated, np.zeros(V.shape[0]), weights=2*np.ones(V.shape[0]), validation_split=None)

1024/1024 [==============================] - 1s 889us/sample - loss: 1.1349 - acc: 0.9824


In [407]:
V = Data.load_random()

In [409]:
generator.predict(V)

array([[ 0.5381674 , -0.7294682 ],
       [-2.823714  ,  2.3865035 ],
       [-1.6322343 , -0.60534656],
       [-1.4527448 , -0.5131934 ],
       [ 1.7400169 , -1.2436492 ],
       [-3.2143238 ,  0.8560622 ],
       [-2.0015056 , -0.10116366],
       [-1.3385688 , -0.6017516 ],
       [-0.63379264, -0.40824658],
       [-0.36235106, -3.1587038 ],
       [ 0.67363346,  0.07549467],
       [-0.51835656, -1.2866111 ],
       [ 1.292838  , -1.5330855 ],
       [ 1.6155298 , -0.34018275],
       [-0.41443527,  0.260574  ],
       [-0.7371965 , -0.5022781 ]], dtype=float32)

In [173]:
gan.Dsc.model.trainable

False

In [174]:
discriminator.model.trainable

False

In [181]:
gan.fit(V, np.ones(16))

InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: You must feed a value for placeholder tensor 'dense_82_input' with dtype float and shape [?,2]
	 [[{{node dense_82_input}}]]
	 [[loss_66/mul/_2913]]
  (1) Invalid argument: You must feed a value for placeholder tensor 'dense_82_input' with dtype float and shape [?,2]
	 [[{{node dense_82_input}}]]
0 successful operations.
0 derived errors ignored.

In [5]:
a = 3
name(a)

NameError: name 'name' is not defined

In [67]:
random.gauss(0.5, 0.3)

0.8464395312821893

In [203]:
discriminator.model.trainable = False

In [204]:
gan.Dsc.model.trainable

False

In [205]:
generator.predict(V)

array([[-0.43098414, -1.8557093 ],
       [-1.416142  ,  0.13706303],
       [-0.9289516 , -1.5910852 ],
       [-0.60927117, -3.3515854 ],
       [ 0.31981492, -0.9599231 ],
       [ 0.70434904, -0.33623135],
       [ 1.1979504 , -1.7666132 ],
       [ 0.33637047, -2.2152154 ],
       [-0.55402684, -1.0059581 ],
       [ 0.7611686 , -2.1785161 ],
       [ 0.17429227, -2.4130015 ],
       [ 1.2904017 , -0.8275587 ],
       [-1.3589315 , -1.6973226 ],
       [-0.9865883 , -1.7392489 ],
       [ 0.33242375,  0.28773135],
       [ 0.9864638 , -0.82728004]], dtype=float32)

In [206]:
V

array([[-9.46154461e-01,  8.42711127e-01,  5.64654642e-01,
        -1.61500519e+00,  2.65035287e-01],
       [-1.29362045e+00,  1.00340807e-02, -1.61517736e+00,
         7.31232946e-04, -6.60969476e-01],
       [ 8.73663091e-01, -1.13003419e+00, -1.56780762e+00,
         1.72668362e+00, -4.57047344e-01],
       [ 2.73782162e+00, -1.68788322e-02, -1.37889423e+00,
         3.61923422e-01,  8.68265394e-03],
       [ 4.05071014e-01, -1.78001050e-01,  6.90572531e-01,
         2.12749241e-01,  1.06033808e+00],
       [-4.49625968e-01, -3.24535215e-01,  1.33955713e-01,
         1.11888694e+00,  6.98768080e-01],
       [ 1.05627802e+00,  1.55713987e-01, -2.57376214e-01,
         1.14346600e+00,  6.36171789e-01],
       [-2.93742272e-02,  2.15808547e+00,  1.71304648e+00,
         8.78535860e-01,  9.64200286e-01],
       [-1.35621070e+00,  1.40061897e+00,  7.00649760e-01,
         4.09271617e-01,  1.54728200e-02],
       [ 9.86079975e-01, -3.52924150e-01, -6.90080659e-01,
         2.33588819e+00

In [236]:
gan.Dsc.__dict__

{'model': <tensorflow.python.keras.engine.sequential.Sequential at 0x24125e4c198>,
 'trainable': False}

In [252]:
discriminator.model.history.history

{'loss': [10496.87605283203], 'acc': [0.5706]}

In [272]:
discriminator.model.trainable

True